<a href="https://colab.research.google.com/github/WoradeeKongthong/raining_tomorrow_classification/blob/master/03_Raining_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Feature Engineering in part 02  
Outliers : I'll cap the outliers in X_train. And cap the outliers in X_test using the boundaries of X_train.  
Missing values : I'll impute the missing values in categorical features with 'most frequent' value,  
and impute the missing values in numerical features with median.

In [0]:
# libraries
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# **Data Set**

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/raining_tomorrow_classification/master/weatherAUS.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [0]:
# drop RISK_MM column (Recommendation from data description in Kaggle)
df.drop(['RISK_MM'], axis = 1, inplace = True)

# Extract Year, Month, Day from Date column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# drop Date column
df.drop(['Date'], axis = 1, inplace = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        141556 non-null  float64
 2   MaxTemp        141871 non-null  float64
 3   Rainfall       140787 non-null  float64
 4   Evaporation    81350 non-null   float64
 5   Sunshine       74377 non-null   float64
 6   WindGustDir    132863 non-null  object 
 7   WindGustSpeed  132923 non-null  float64
 8   WindDir9am     132180 non-null  object 
 9   WindDir3pm     138415 non-null  object 
 10  WindSpeed9am   140845 non-null  float64
 11  WindSpeed3pm   139563 non-null  float64
 12  Humidity9am    140419 non-null  float64
 13  Humidity3pm    138583 non-null  float64
 14  Pressure9am    128179 non-null  float64
 15  Pressure3pm    128212 non-null  float64
 16  Cloud9am       88536 non-null   float64
 17  Cloud3pm       85099 non-null

In [0]:
X = df.drop(['RainTomorrow'], axis=1)
y = df['RainTomorrow']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [8]:
# create temp_df to combine X_train and y_train and cap the outliers
temp_df = X_train
temp_df['RainTomorrow'] = y_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
temp_df

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,Year,Month,Day,RainTomorrow
60146,Sale,16.8,20.7,31.2,2.4,0.3,ESE,39.0,SE,SE,24.0,26.0,97.0,73.0,1014.2,1015.3,8.0,8.0,17.5,20.4,Yes,2010,2,15,Yes
83290,Brisbane,23.1,26.1,39.2,1.6,0.0,NE,44.0,ENE,E,17.0,15.0,92.0,91.0,1008.1,1006.0,8.0,8.0,24.9,25.1,Yes,2013,1,26,Yes
106134,Albany,7.4,21.2,0.0,1.6,8.8,NaN,NaN,NaN,SE,0.0,6.0,89.0,69.0,1022.7,1020.0,0.0,1.0,9.4,19.9,No,2009,5,16,No
42732,Wollongong,17.2,28.7,NaN,NaN,NaN,NW,69.0,NW,NNW,33.0,13.0,32.0,73.0,1006.5,1001.1,NaN,1.0,26.4,20.6,NaN,2012,11,1,No
5938,BadgerysCreek,0.8,18.6,0.2,NaN,NaN,WSW,31.0,NaN,WSW,0.0,20.0,99.0,40.0,1018.8,1015.5,NaN,NaN,6.9,17.8,No,2017,6,25,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23187,NorfolkIsland,20.4,24.5,0.0,3.0,9.3,SSE,31.0,SSE,ESE,20.0,17.0,81.0,74.0,1019.7,1017.6,4.0,3.0,22.5,22.9,No,2016,4,14,No
41133,Williamtown,12.4,25.9,0.0,NaN,NaN,NaN,NaN,WNW,S,28.0,33.0,45.0,72.0,1015.5,1015.8,8.0,6.0,21.7,21.1,No,2016,10,8,Yes
904,Albury,0.2,15.2,0.0,NaN,NaN,N,20.0,E,SE,4.0,9.0,99.0,63.0,1026.7,1020.9,7.0,NaN,3.7,14.5,No,2011,7,2,No
44601,Canberra,2.9,9.5,16.8,1.4,6.5,NW,35.0,NaN,NW,0.0,15.0,93.0,72.0,1021.6,1019.6,6.0,6.0,5.5,7.4,Yes,2008,7,8,No


In [0]:
# cap the outliers in Training set

Q1 = temp_df.quantile(0.25)
Q3 = temp_df.quantile(0.75)
IQR = Q3 - Q1

lower_cap = Q1 - 1.5*IQR
upper_cap = Q3 + 1.5*IQR

features = lower_cap.index.values

for feature in features :
  temp_df[feature] = np.where(temp_df[feature]<lower_cap[feature],lower_cap[feature], temp_df[feature])
  temp_df[feature] = np.where(temp_df[feature]>upper_cap[feature],upper_cap[feature], temp_df[feature])

In [0]:
# get y_train and X_train from temp_df
X_train = temp_df.drop(['RainTomorrow'], axis=1)
y_train = temp_df['RainTomorrow']

In [0]:
# cap outliers on the test set
for feature in features :
  X_test[feature] = np.where(X_test[feature]<lower_cap[feature],lower_cap[feature], X_test[feature])
  X_test[feature] = np.where(X_test[feature]>upper_cap[feature],upper_cap[feature], X_test[feature])

# **Create Model Pipeline**

**Create Preprocessor : ColumnTransformer of numerical and categorical features**

In [0]:
numerical_features = [x for x in X.columns if df[x].dtype != 'object']

numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', MinMaxScaler())
])

categorical_features = [x for x in X.columns if df[x].dtype == 'object']

categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
          ('num', numeric_transformer, numerical_features),
          ('cat', categorical_transformer, categorical_features)
    ]
)

**Create model**

In [0]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2, n_jobs = -1)

**Create Pipeline**

In [0]:
clf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('model', model)
])

# **Cross Validation on Training set**

In [16]:
accuracy = cross_val_score(clf,X_train,y_train,cv=10)
print('accuracy : ', accuracy)
print('mean : ', accuracy.mean())
print('std : ', accuracy.std())

accuracy :  [0.80863221 0.80537975 0.80344585 0.80617089 0.80413187 0.80202198
 0.81081319 0.80386813 0.79964835 0.8       ]
mean :  0.8044112208466638
std :  0.0033465439298328153


Note : low bias and very low variance

# **Training and Test sets**

In [17]:
# Make sure your X_test is capped before running this cell

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print('\n\nTraining and Test Sets result')
print('\naccuracy score : ', accuracy_score(y_test,y_pred))
print('\nconfusion matrix : \n', confusion_matrix(y_test, y_pred))
print('\nclassification report : \n', classification_report(y_test,y_pred))

print('Training set score : ',clf.score(X_train,y_train))
print('Test set score : ',clf.score(X_test,y_test))



Training and Test Sets result

accuracy score :  0.8008720419142726

confusion matrix : 
 [[20298  1549]
 [ 4114  2478]]

classification report : 
               precision    recall  f1-score   support

          No       0.83      0.93      0.88     21847
         Yes       0.62      0.38      0.47      6592

    accuracy                           0.80     28439
   macro avg       0.72      0.65      0.67     28439
weighted avg       0.78      0.80      0.78     28439

Training set score :  0.8577017071927141
Test set score :  0.8008720419142726


Note : there is overfitting in the model

CON : training KNN model takes long time